In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
961,I don't see why all the people are giving this...,positive
165,A widely unknown strange little western with m...,positive
86,Two years ago at Sundance I loved Josh Kornblu...,negative
18,There are other reviews here expressing simila...,negative
493,I was never all that impressed by Night Galler...,positive


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)
df.head()

Error during text normalization: 
**********************************************************************
  Resource wordnet not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('wordnet')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/wordnet

  Searched in:
    - 'C:\\Users\\SOUMEN/nltk_data'
    - 'c:\\Users\\SOUMEN\\anaconda3\\envs\\atlas\\nltk_data'
    - 'c:\\Users\\SOUMEN\\anaconda3\\envs\\atlas\\share\\nltk_data'
    - 'c:\\Users\\SOUMEN\\anaconda3\\envs\\atlas\\lib\\nltk_data'
    - 'C:\\Users\\SOUMEN\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************



LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\SOUMEN/nltk_data'
    - 'c:\\Users\\SOUMEN\\anaconda3\\envs\\atlas\\nltk_data'
    - 'c:\\Users\\SOUMEN\\anaconda3\\envs\\atlas\\share\\nltk_data'
    - 'c:\\Users\\SOUMEN\\anaconda3\\envs\\atlas\\lib\\nltk_data'
    - 'C:\\Users\\SOUMEN\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [7]:
df['sentiment'].value_counts()

sentiment
positive    250
negative    250
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
961,see people giving film negative reviews loved ...,1
165,widely unknown strange little western mindblow...,1
86,two years ago sundance loved josh kornbluth s ...,0
18,reviews expressing similar views still feel im...,0
493,never impressed night gallery one episode stan...,1


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [11]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Samacker25/Capstone-project-1.mlflow')
dagshub.init(repo_owner='Samacker25', repo_name='Capstone-project-1', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-11-25 15:31:22,587 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Samacker25/Capstone-project-1 "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Samacker25/Capstone-project-1"

2025-11-25 15:31:22,602 - INFO - Initialized MLflow to track repo "Samacker25/Capstone-project-1"


Repository Samacker25/Capstone-project-1 initialized!

2025-11-25 15:31:22,610 - INFO - Repository Samacker25/Capstone-project-1 initialized!


<Experiment: artifact_location='mlflow-artifacts:/131fa35c6e3045b1b6a8311eff1946b1', creation_time=1764052890220, experiment_id='0', last_update_time=1764052890220, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-25 15:33:06,808 - INFO - Starting MLflow run...
2025-11-25 15:33:07,269 - INFO - Logging preprocessing parameters...
2025-11-25 15:33:08,701 - INFO - Initializing Logistic Regression model...
2025-11-25 15:33:08,701 - INFO - Fitting the model...
2025-11-25 15:33:08,752 - INFO - Model training complete.
2025-11-25 15:33:08,752 - INFO - Logging model parameters...
2025-11-25 15:33:09,129 - INFO - Making predictions...
2025-11-25 15:33:09,135 - INFO - Calculating evaluation metrics...
2025-11-25 15:33:09,146 - INFO - Logging evaluation metrics...
2025-11-25 15:33:11,158 - INFO - Saving and logging the model...
2025/11/25 15:33:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-11-25 15:33:12,167 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\SOUMEN\AppData\Local\Temp\ipykernel_7776\3279679509.py", line 48, in 

🏃 View run honorable-flea-487 at: https://dagshub.com/Samacker25/Capstone-project-1.mlflow/#/experiments/0/runs/096c8fea20fa4a679a9df23ed5ce1f5d
🧪 View experiment at: https://dagshub.com/Samacker25/Capstone-project-1.mlflow/#/experiments/0
